# COGS 108 - Final Project

## Important

- ONE, and only one, member of your group should upload this notebook to TritonED. 
- Each member of the group will receive the same grade on this assignment. 
- Keep the file name the same: submit the file 'FinalProject.ipynb'.
- Only upload the .ipynb file to TED, do not upload any associted data. Make sure that for cells in which you want graders to see output that these cells have been executed.

## Group Members: Fill in the Student IDs of each group member here

Replace the lines below to list each persons full student ID, ucsd email and full name.

-  
- 
- 
- 



# Part 0 - Importing the required packages

In [5]:
import pandas as pd # for use of dataframes
import numpy as np
import matplotlib.pyplot as plt # for plotting graphs 
import statsmodels.api as sm # for fitting an OLS model
import scipy.stats as stats # for access to tests for normal distribution
from scipy.stats import ttest_ind, normaltest # for access to t tests, normal test

# Part 1 - Introduction and Background

Chicago, otherwise known as 'The Windy City', has some of the worst crime rates in America. Some of this has to do with statitical flukes; it is the third largest city in the U.S., which makes any type of criminal activity more easily reportable. However, most people wouldn't argue over the real danger you are in when you go to some of the worse off areas of Chicago.

We wanted to analyze the crime rates in Chicago in some way in order to assist people in the city in understanding what problems exist and where they are happening. In the end, we decided to ask the following question:

"Does the district a crime is committed in or type of crime more closely predict whether an arrest was made".

The answers derived from the answer to this question could spawn further analysis, e.g.:

1. If a particular crime is more likely to result in an arrest in one district over the other, why is that? Is it the police in that district? Is it the citizens? Or is it the geography?

2. If the type of crime affects whether an arrest is made, does that mean that we are more afraid of certain crimes over others? Is this a valid fear? 

# Part 2 - Data Description

We are using this dataset: https://www.kaggle.com/chicago/chicago-crime

This is chicago crime data from 2001 to present. It represents all reported instances of crime, and whether or not an arrest has been made since the data was last released. Each row represents a crime, except for multiple homicides, for which there is a different row for each victim of the crime. 

Here are some basic stats:

In [29]:
crime = pd.read_csv('~/data/chicago_crime_data.csv')

In [7]:
crime.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,6.802359e+06,6.802359e+06,6.802312e+06,6.187505e+06,6.188828e+06,6.740394e+06,6.740394e+06,6.802359e+06,6.740394e+06,6.740394e+06
mean,6.267879e+06,1.191551e+03,1.130251e+01,2.268313e+01,3.758620e+01,1.164517e+06,1.885721e+06,2.008401e+03,4.184201e+01,-8.767180e+01
std,3.063983e+06,7.033569e+02,6.945669e+00,1.383246e+01,2.154045e+01,1.716683e+04,3.270468e+04,5.067564e+00,8.999481e-02,6.212430e-02
min,6.340000e+02,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01
25%,3.443594e+06,6.220000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.152934e+06,1.859189e+06,2.004000e+03,4.176892e+01,-8.771387e+01
50%,6.254311e+06,1.111000e+03,1.000000e+01,2.200000e+01,3.200000e+01,1.165991e+06,1.890586e+06,2.008000e+03,4.185537e+01,-8.766618e+01
75%,8.909085e+06,1.731000e+03,1.700000e+01,3.400000e+01,5.800000e+01,1.176352e+06,1.909313e+06,2.012000e+03,4.190686e+01,-8.762836e+01
max,1.158934e+07,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.019000e+03,4.202291e+01,-8.752453e+01


In [9]:
print('Total Number or Crimes:\t\t\t{0}'.format(len(crime)))
print('Total number of Unique Districts:\t{0}'.format(len(crime['District'].unique())))

Total Number or Crimes:			6802359
Total number of Unique Districts:	25


# Part 3 - Data Cleaning and Preprocessing

This data is fairly clean, because it is maintained by the Chicago Police Department, which has high organizational standards. There aren't very many invalid crimes or unknowns.

To clean the data, we will do the following (not necessarily in order):

1. Drop useless columns
2. Standardize the location description
3. Standardize the crime type to get rid of the uppercase, standardize the non-criminal type
4. Remove NaN values

In [30]:
# first, drop the location description column and community area column because 
crime = crime.drop(['Location Description','Community Area','Latitude','Longitude','X Coordinate','Y Coordinate','Location','Ward'],axis=1)

In [31]:
def standardize_primary_type(string):
    # compile all non-criminal offenses into on label
    if string == 'NON-CRIMINAL (SUBJECT SPECIFIED)' or string == 'NON - CRIMINAL' or string == 'NON-CRIMINAL':
        return 'non-criminal'
    if string == 'OTHER OFFENSE':
        return 'other'
    # rename crim sexual assault to just sexual assault to make it easier to read
    if string == 'CRIM SEXUAL ASSAULT':
        return 'sexual assault'
    else:
        # everything else, make sure to lowercase it so we don't have to use caps lock lol
        return string.lower()
        

In [32]:
# Standardize the crime type
crime['Type'] = crime['Primary Type'].apply(standardize_primary_type)

In [33]:
crime[crime.isnull().any(axis=1)]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Arrest,Domestic,Beat,District,FBI Code,Year,Updated On,Type
708934,11181991,JA549530,12/14/2017 08:10:00 PM,100XX W BRYN MAWR AVE,0460,BATTERY,SIMPLE,False,False,1654,NaN,08B,2017,05/04/2018 03:51:04 PM,battery
1314608,4437079,HL727137,11/10/2005 10:10:00 AM,010XX E GRAND AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,False,True,1834,NaN,26,2005,02/10/2018 03:50:01 PM,other
1813672,6376239,HP461051,07/18/2008 08:00:00 PM,010XX N SPRINGFIELD AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,False,False,1112,NaN,14,2008,02/10/2018 03:50:01 PM,criminal damage
1821722,6420740,HP503796,01/07/2007 05:00:00 AM,030XX S ASHLAND AVE,1140,DECEPTIVE PRACTICE,EMBEZZLEMENT,False,False,922,NaN,12,2007,02/10/2018 03:50:01 PM,deceptive practice
1829184,6462999,HP539483,08/23/2008 12:00:00 PM,061XX S MICHIGAN AVE,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,False,False,311,NaN,10,2008,02/10/2018 03:50:01 PM,deceptive practice
1838356,6516041,HP587362,09/22/2008 09:10:00 PM,013XX N CALIFORNIA AVE,0560,ASSAULT,SIMPLE,False,False,1423,NaN,08A,2008,02/10/2018 03:50:01 PM,assault
1841841,6534030,HP607574,09/03/2008 12:00:00 PM,003XX N WOLCOTT AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,False,False,1333,NaN,07,2008,02/10/2018 03:50:01 PM,motor vehicle theft
1846326,6555854,HP629252,10/15/2008 04:10:00 PM,021XX W NORTH AVE,0610,BURGLARY,FORCIBLE ENTRY,False,False,1424,NaN,05,2008,02/10/2018 03:50:01 PM,burglary
1847499,6562039,HP633798,10/17/2008 10:39:00 PM,025XX W HUTCHINSON ST,0320,ROBBERY,STRONGARM - NO WEAPON,False,False,1912,NaN,03,2008,02/10/2018 03:50:01 PM,robbery
1850390,6576124,HP528527,08/21/2008 02:30:00 PM,004XX N MC CLURG CT,1152,DECEPTIVE PRACTICE,ILLEGAL USE CASH CARD,False,False,1834,NaN,11,2008,02/10/2018 03:50:01 PM,deceptive practice


# Part 4 - Data Visualization

# Part 5 - Data Analysis and Results

# Part 6 - Privacy/Ethics Considerations

The data is private because there is no identifying information. There are no names, only crime ids and description. The only identifying information is the location. This could be an issue because some of the crimes occurred in apartments, so you could conceivably find out the location where a crime was committed and use this to find someone.

A potential ethics considerations involving this data is 

# Part 7 - Conclusions and Discussion